# Analyse des données après fusion des tables

Le script preprocessing.py génère un tableau de données unique avec toutes les features disponibles. Chaque ligne correspond à un client unique. Les colonnes contiennent toutes des données quantitatives.

Comme le fichier de données est trop gros et provoque des crashs dans Jupyter notebook, il convient de le séparer en train et test et de n'utiliser que le train set pour entraîner un modèle de classification et sélectionner les variables.

In [3]:
import pandas as pd
import gc

https://stackoverflow.com/questions/845058/how-to-get-line-count-of-a-large-file-cheaply-in-python

In [4]:
# recherche du nombre de lignes d'un fichier csv sans l'ouvrir
sum(1 for line in open('../data/final_credit.csv'))

356256

https://stackoverflow.com/questions/845058/how-to-get-line-count-of-a-large-file-cheaply-in-python

On peut utiliser dask pour charger le dataset mais compliqué car pas exactement un Dataframe exploitable avec Pandas

La solution retenue est celle d'une lecture du fichier csv par lots  
https://stackoverflow.com/questions/25962114/how-do-i-read-a-large-csv-file-with-pandas

In [6]:
# Ouverture du fichier csv en plusieurs lots afin d'éviter des plantages
TextFileReader = pd.read_csv("../data/final_credit.csv",
                             chunksize=10000, index_col=0)  # the number of rows per chunk

dfList = []
for df in TextFileReader:
    dfList.append(df)

df = pd.concat(dfList,sort=False)

In [7]:
df.shape

(356255, 798)

In [8]:
df.tail()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
356250,48739,456221,NaN,1,0,0,0,121500.0,412560.0,17473.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356251,48740,456222,NaN,1,0,1,2,157500.0,622413.0,31909.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356252,48741,456223,NaN,1,1,0,1,202500.0,315000.0,33205.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356253,48742,456224,NaN,0,0,1,0,225000.0,450000.0,25128.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356254,48743,456250,NaN,1,1,1,0,135000.0,312768.0,24709.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0


Du fait de l'encodage One Hot des variables catégorielles, le nombre de colonnes augmente fortement.

In [9]:
# Certaines colonnes contiennent beaucoup de valeurs manquantes
df.isnull().sum()

index                                    0
SK_ID_CURR                               0
TARGET                               48744
CODE_GENDER                              0
FLAG_OWN_CAR                             0
                                     ...  
CC_NAME_CONTRACT_STATUS_nan_MAX     252697
CC_NAME_CONTRACT_STATUS_nan_MEAN    252697
CC_NAME_CONTRACT_STATUS_nan_SUM     252697
CC_NAME_CONTRACT_STATUS_nan_VAR     253389
CC_COUNT                            252697
Length: 798, dtype: int64

In [10]:
# Suppression de la colonne 'index' et réinitialisation de l'index
df.drop(columns=['index'], inplace=True)

In [11]:
df = df.reset_index(drop=True)

In [12]:
df.loc[:10,:].dropna(subset=['TARGET'])

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,100008,0.0,0,0,0,0,99000.0,490495.5,27517.5,454500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,100009,0.0,1,1,0,1,171000.0,1560726.0,41301.0,1395000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,100010,0.0,0,1,0,0,360000.0,1530000.0,42075.0,1530000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,100011,0.0,1,0,0,0,112500.0,1019610.0,33826.5,913500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0
9,100012,0.0,0,0,0,0,135000.0,405000.0,20250.0,405000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Le test set correspond aux lignes sans target
df_test = df[df['TARGET'].isna()]

In [14]:
# Enregistrement des données du test set
df_test.to_csv('../data_models/credit_test.csv')

In [15]:
# Supression du test set pour éviter les problèmes mémoire
del df_test
gc.collect()

0

In [16]:
# isoler les données du train set
df_train = df.dropna(subset=['TARGET'])

In [17]:
df_train.shape

(307511, 797)

In [18]:
# Enregistrement des données du train set
df_train.to_csv('../data_models/credit_train.csv')

In [19]:
del df
gc.collect()

0

In [20]:
# Vérifier que SK_ID_CURR est bien un identifiant unique
df_train.SK_ID_CURR.nunique()

307511

# Analyse de la répartition des classes dans le train set

In [21]:
# Sélection des lignes ayant une target = 1 (crédit refusé)
df_target1 = df_train[df_train["TARGET"]==1]

In [22]:
# Sélection des lignes ayant une target = 1 (crédit accordé)
df_target0 = df_train[df_train["TARGET"]==0]

In [23]:
len(df_target1)

24825

In [24]:
len(df_target0)

282686

On voit qu'il y a un gros déséquilibre entre les classes. Il y a une minorité de crédits refusés (target 1).

In [25]:
# Annalyse du pourcentage de valeurs manquantes
na_df = (df_train.isnull().sum()/len(df_target0)).sort_values(ascending=False)
na_df

REFUSED_AMT_DOWN_PAYMENT_MAX      0.92803
REFUSED_AMT_DOWN_PAYMENT_MEAN     0.92803
REFUSED_AMT_DOWN_PAYMENT_MIN      0.92803
REFUSED_RATE_DOWN_PAYMENT_MEAN    0.92803
REFUSED_RATE_DOWN_PAYMENT_MAX     0.92803
                                   ...   
ORGANIZATION_TYPE_Government      0.00000
ORGANIZATION_TYPE_Emergency       0.00000
ORGANIZATION_TYPE_Electricity     0.00000
ORGANIZATION_TYPE_Culture         0.00000
SK_ID_CURR                        0.00000
Length: 797, dtype: float64

Certaines colonnes ont plus de 90% de valuers manquantes.

In [26]:
# Liste des variables ayant moins de 10% de valeurs manquantes
na_df[na_df<0.1]

APPROVED_AMT_GOODS_PRICE_MEAN           0.065178
APPROVED_AMT_GOODS_PRICE_MAX            0.065178
APPROVED_AMT_GOODS_PRICE_MIN            0.065178
POS_NAME_CONTRACT_STATUS_Active_MEAN    0.063912
POS_SK_DPD_DEF_MEAN                     0.063912
                                          ...   
ORGANIZATION_TYPE_Government            0.000000
ORGANIZATION_TYPE_Emergency             0.000000
ORGANIZATION_TYPE_Electricity           0.000000
ORGANIZATION_TYPE_Culture               0.000000
SK_ID_CURR                              0.000000
Length: 442, dtype: float64